In [14]:
import torch                                                                                          
                                                                                                      
def jacobian(y, x, create_graph=False):                                                               
    jac = []                                                                                          
    flat_y = y.reshape(-1)                                                                            
    grad_y = torch.zeros_like(flat_y)  
    # for each function output
    for i in range(len(flat_y)):                                                                      
        grad_y[i] = 1.                                                                                
        grad_x, = torch.autograd.grad(flat_y, x, grad_y, retain_graph=True, create_graph=create_graph)
        jac.append(grad_x.reshape(x.shape))                                                           
        grad_y[i] = 0.                                                                                
    return torch.stack(jac).reshape(y.shape + x.shape)                                                
                                                                                                      
def hessian(y, x):                                                                                    
    return jacobian(jacobian(y, x, create_graph=True), x)                                             
                                                                                                      
def f(x):                                                                                             
    return x * x * torch.arange(4, dtype=torch.float)   

def rosenbrock(x):
    assert len(x) == 2
    return (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2 

def rosenbrock_jacobian(x):
    return torch.tensor([
        -400*x[0]*x[1] + 400*x[0]**3+2*x[0]-2,
        200*x[1] - 200 * x[0]
    ])
def rosenbrock_hessian(x):
    return torch.tensor([
        [-400 * x[1] + 1200* x[0]**3 + 2,   -400*x[0]],
        [-400*x[0]                      , 200]
    ])  

In [15]:
x = torch.tensor([0., 0.], requires_grad=True)  
y = rosenbrock(x)

print('analytical grad:', rosenbrock_jacobian(x))
print('autograd grad:', jacobian(y, x))

print('analytical hess:', rosenbrock_hessian(x))
print('autograd hess:', hessian(y, x))

analytical grad: tensor([-2.,  0.])
autograd grad: tensor([-2.,  0.])
analytical hess: tensor([[  2.,  -0.],
        [ -0., 200.]])
autograd hess: tensor([[  2.,   0.],
        [  0., 200.]])
